In [1]:
print("hii")

hii


In [5]:
pip install pymupdf ctransformers llama-index llama-cpp-python sentence-transformers transformers

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached ctransformers-0.2.27-py3-none-any.whl.metadata (17 kB)
     ---------------------------------------- 0.0/50.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/50.6 MB ? eta -:--:--
     ---------------------------------------- 0.3/50.6 MB ? eta -:--:--
     ---------------------------------------- 0.5/50.6 MB 1.1 MB/s eta 0:00:45
      --------------------------------------- 1.0/50.6 MB 1.5 MB/s eta 0:00:34
     - -------------------------------------- 1.3/50.6 MB 1.5 MB/s eta 0:00:33
     - -------------------------------------- 1.8/50.6 MB 1.7 MB/s eta 0:00:30
     - -------------------------------------- 2.4/50.6 MB 1.8 MB/s eta 0:00:27
     -- ------------------------------------- 2.9/50.6 MB 1.9 MB/s eta 0:00:26
     -- ------------------------------------- 3.1/50.6 MB 1.9 MB/s eta 0:00:25
     -- ---------

ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:

from transformers import pipeline, AutoTokenizer
import time
import pymupdf
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from ctransformers import AutoModelForCausalLM
from llama_index.core import PromptTemplate#used to creata prompt template
from llama_cpp import Llama
import json

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
def dynamic_threshold(query_text):
    if len(query_text.split()) <= 6:
        return 0.5  # short query → use low threshold
    elif len(query_text.split()) <= 12:
        return 0.6
    else:
        return 0.754

In [ ]:

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def smart_chunk_text(text, max_chars=800):
    sentences = sent_tokenize(text)
    chunks, chunk = [], ""
    for sentence in sentences:
        if len(chunk) + len(sentence) <= max_chars:
            chunk += " " + sentence
        else:
            chunks.append(chunk.strip())
            chunk = sentence
    if chunk:
        chunks.append(chunk.strip())
    return chunks

def extract_paragraphs_from_pdf(pdf_path):
    doc = pymupdf.open(pdf_path)
    all_text = ""
    for page in doc:
        all_text += page.get_text()
    paragraphs = smart_chunk_text(all_text)
    return paragraphs


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:

from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')  # lightweight and local-friendly

def rerank(query, passages):
    pairs = [(query, passage) for passage in passages]
    scores = cross_encoder.predict(pairs)
    sorted_passages = [p for _, p in sorted(zip(scores, passages), reverse=True)]
    return sorted_passages


In [6]:

def build_llm_prompt(query, clauses):
    prompt_template = """Query: {query}
Relevant Clauses:
{clauses}

Instructions:
Based on the query and clauses, return a JSON with:
- decision: Approved or Rejected
- amount: if mentioned
- justification: list of clause-based reasoning

Respond ONLY with JSON.
"""
    return prompt_template.format(query=query, clauses="\n\n".join(clauses))


In [ ]:
pdf_path='BAJHLIP23020V012223.pdf'
model_path="models"
embedding_path='stronger_embedding_model'

In [ ]:
def extract_paragraphs_from_pdf(pdf_path):
    doc = pymupdf.open(pdf_path)
    all_text = ""

    for page in doc:
        all_text += page.get_text()
    paragraphs = re.split(r'\n\s*\n', all_text)

    
    return paragraphs

In [9]:
summarizer = pipeline("summarization", model=model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model=SentenceTransformer(embedding_path)

Device set to use cpu


In [10]:
paragraph=extract_paragraphs_from_pdf(pdf_path)
answers=[]
datas=[]

In [11]:
start=time.time()
for i in range(len(paragraph)):
    para = paragraph[i].strip()
    if len(para)<10:
        continue

    # # print(f"\nParagraph {i + 1}:")
    # print(para, "\n")

    #token leng
    tokens = tokenizer(para, return_tensors="pt", truncation=True)
    input_len = tokens.input_ids.shape[1]

    # Dynamically set summarization length
    max_len = int(input_len * 0.8)
    min_len = max(10, int(input_len * 0.5))

    # Ensure max_len is shorter than input_len
    max_len = min(max_len, input_len - 1)

    summary = summarizer(para, max_length=max_len, min_length=min_len, num_beams=1, do_sample=False)[0]['summary_text']
    answers.append(summary)
    datas.append(para)

    # print(f"Summary:\n{summary}")
    print(f"Time elapsed: {time.time() - start:.2f} seconds")

The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Time elapsed: 1.43 seconds


Your min_length=10 must be inferior than your max_length=4.
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
d:\free_python\.venv\Lib\site-packages\transformers\generation\utils.py:1640: UserWarning: Unfeasible length constraints: `min_length` (10) is larger than the maximum possible length (4). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Time elapsed: 3.65 seconds
Time elapsed: 3.78 seconds


...

The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Time elapsed: 611.85 seconds


Token indices sequence length is longer than the specified maximum sequence length for this model (1084 > 1024). Running this sequence through the model will result in indexing errors
The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Time elapsed: 612.45 seconds


IndexError: index out of range in self

In [12]:

# embedding_map={ [(data:[],embeddings=[])] * whatever }
#ill be appending the data like this: (para data,embedding)\

# print(answers[4])
# print("---------------")
# print(answers[5])
# emb=model.encode(answers[4])
# emb2=model.encode(answers[5])
# print(cosine_similarity([emb],[emb2])[0][0])
summaries=answers
#i also have all the parag data in the datas arr
embedding_map={}
THRESHOLD=0.7
group_index = 0

for i in range(len(summaries)):
    embed=[model.encode(summaries[i])] #im only encoding summaries rn, maybe ill do datas only later or both idk
    matched=False

    for j in range(len(embedding_map)):
        summarized_data,embedding,para_data=embedding_map[j][0]
        sim=cosine_similarity(embedding,embed)[0][0]
        if sim>THRESHOLD:
            embedding_map[j].append( (summaries[i],embed,datas[i]) )
            matched = True
    if not matched:
        embedding_map[group_index] = [ ( summaries[i],embed,datas[i] ) ]
        group_index+=1



d:\free_python\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
d:\free_python\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
d:\free_python\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
d:\free_python\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
d:\free_python\.venv\Lib\site-packages\t

In [15]:
for i in range(3,len(embedding_map)):
    print(f"\n.Group {i + 1}:")
    for para, _,data in embedding_map[i]:
        print("-", para)


.Group 4:
-  The Policy is based on the Standard Definitions and Specific Definitions of words and terms . The Policy will be amended to reflect the meaning of the words and phrases mentioned below . The policies are based on these definitions and the specific meanings of words . The policy is based upon the Standard Definition and Specific definitions of words or terms .
-  Endorsement means any writing on a Policy Schedule or Policy, in addition to its normal Policy Schedule/Policy wording/Standard Terms and Conditions . It may be added when Policy is prepared, or subsequently . Provided any Service Level Agreement [SLA] or Agreement/MOU laying down various service levels shall not be treated as Endorsements .
-  Policy or Contract means the Proposal, the Policy Schedule, along with these Terms and Conditions issued to the Insured . Policy or contract means the proposal, the policy Schedule, the . Policy Schedule and these Terms of the  Insured and any annexures and/or Endorse
-  Po

In [ ]:
def create_natural_language_llm():
    # llm = AutoModelForCausalLM.from_pretrained(
    #     'models\phi-2.Q4_K_M.gguf',
    #     model_type='llama',
    #     context_length=2048
    # )
    print("starting")
    llm = Llama(
        # model_path='models\openhermes-2.5-mistral-7b.Q5_K_M.gguf',
        model_path='tinyllama_model\\tinyllama-1.1b-chat-v1.0.Q5_K_M.gguf',
        # model_path='models\gemma-2-2b-it-f32.gguf',
        # model_path='models\gemma-2-2b-it-Q4_K_M.gguf',
        # model_path='models/phi-2.Q4_K_M.gguf',
        n_ctx=2048,        # Keep small unless you need long context
        n_threads=5,       # Match your CPU's cores
        n_batch=256 ,       # Speed up generation
        f16_kv=True,       # Lower memory, faster KV cache
        use_mlock=False,    # Pin model in RAM (optional, speeds up access)
        verbose=False
    )
    # print(llm.config["context_length"])
    return llm

def get_embedding_friendly_prompt():
    prompt = '''### Instruction:
Write a concise and short answer to the query given below using the data in Paragraph. ANSWER IN SIMPLE YES OR NO AND FOLLOW ALONG WITH THE QUERY
User query:
{user_query}
Paragraph:
{paragraph}

### Response:'''
    return PromptTemplate(prompt)

In [17]:
#so now i need to have a query and match similarity for each group
llm=create_natural_language_llm()

query='46M, knee surgery, Pune, 3-month policy'
query_embed= [ model.encode(query) ]
prompt_data=[]
for i in range(len(embedding_map)):
    # for j in range(len(embedding_map[i])):
    summarized_data,embedding,para_data=embedding_map[i][0]
    sim=cosine_similarity(embedding,query_embed)[0][0]
    if sim>dynamic_threshold(query):
        prompt_data.append(summarized_data)
    # if sim>0.7:
    #     prompt_data.append(summarized_data)

# prompt=get_embedding_friendly_prompt()
# prompt=prompt.format(
#     user_query=query,paragraph=json.dumps(prompt_data)
# )
# para = 'Write this in short '+query.strip()
# if len(para)>10:
# # # print(f"\nParagraph {i + 1}:")
# # print(para, "\n")
#     #token leng
#     tokens = tokenizer(para, return_tensors="pt", truncation=True)
#     input_len = tokens.input_ids.shape[1]

#     # Dynamically set summarization length
#     max_len = int(input_len * 0.8)
#     min_len = max(10, int(input_len * 0.5))

#     # Ensure max_len is shorter than input_len
#     max_len = min(max_len, input_len - 1)

#     summary = summarizer(para, max_length=max_len, min_length=min_len, num_beams=1, do_sample=False)[0]['summary_text']
#     print(summary)
# query_embed=[ model.encode(query) ]
# answers=[]
# for i in range(len(embedding_map)):
#     data,embedding=embedding_map[i][0]
#     sim=cosine_similarity(embedding,query_embed)[0][0]
#     # print(data)
#     if sim>0.7:
#         answers.append(data)
#         print(data)

# for ans in answers:
#     print(ans)



starting


llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility
d:\free_python\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [18]:
# print(len(prompt_data))

# print(len(answers))
prompt_data

[' UIN- BAJHLIP2302020V012223 . UIN:    BAJ HLIP230 20V0123 .   UIN 23020V022223. U',
 ' Bajaj Allianz General Insurance Co. Ltd. is a Pune-based insurance company . The company is based in the city of Yerawada, Pune . The Pune based company is a global health insurance provider . The cost of the insurance is based at Bajaja Allianza General Insurance Company, Puna .',
 ' Policy',
 ' Any one Illness means continuous Period of Illness . It includes relapse within 45 days from the date of last consultation with the Hospital/Nursing Home where treatment was taken . 2',
 ' AYUSH Hospital is a healthcare facility where medical/surgical/para-surgical treatment procedures and interventions are carried out by AYush Medical Practitioner(s) It must comply with all the following criterion:  Having at least 5 Inpatient beds; \xa0having qualified AYUS Medical Practitionser in charge round the clock;  Having dedicated AYushes therapy sections as required and/or has dedicated AyUSH therapy sections .

In [19]:
summarizer_text = f"Can you summarize the following content based on the query: '{query}'\n\n{'\n'.join(prompt_data)}"

In [20]:
# para = summarizer_text.strip()
# if len(para)>10:
# #token leng
#     tokens = tokenizer(para, return_tensors="pt", truncation=True)
#     input_len = tokens.input_ids.shape[1]

#     # Dynamically set summarization length
#     max_len = int(input_len * 0.7)
#     min_len = max(10, int(input_len * 0.5))

#     # Ensure max_len is shorter than input_len
#     max_len = min(max_len, input_len - 1)

#     summary = summarizer(summarizer_text, max_length=max_len, min_length=min_len, num_beams=1, do_sample=False)[0]['summary_text']


# Truncate total characters or paragraphs if it's too long
max_para_char_limit = 4000  # adjust based on tokenizer type
joined_data = "\n".join(prompt_data)
if len(joined_data) > max_para_char_limit:
    joined_data = joined_data[:max_para_char_limit]

summarizer_text = f"Can you summarize the following content based on the query: '{query}'\n\n{joined_data}"


In [21]:
# len(summarizer_text)
summarizer_text

"Can you summarize the following content based on the query: '46M, knee surgery, Pune, 3-month policy'\n\n UIN- BAJHLIP2302020V012223 . UIN:    BAJ HLIP230 20V0123 .   UIN 23020V022223. U\n Bajaj Allianz General Insurance Co. Ltd. is a Pune-based insurance company . The company is based in the city of Yerawada, Pune . The Pune based company is a global health insurance provider . The cost of the insurance is based at Bajaja Allianza General Insurance Company, Puna .\n Policy\n Any one Illness means continuous Period of Illness . It includes relapse within 45 days from the date of last consultation with the Hospital/Nursing Home where treatment was taken . 2\n AYUSH Hospital is a healthcare facility where medical/surgical/para-surgical treatment procedures and interventions are carried out by AYush Medical Practitioner(s) It must comply with all the following criterion:  Having at least 5 Inpatient beds; \xa0having qualified AYUS Medical Practitionser in charge round the clock;  Having 

In [22]:
para = summarizer_text.strip()
if len(para)>10:
#token leng
    tokens = tokenizer(para, return_tensors="pt", truncation=True)
    input_len = tokens.input_ids.shape[1]

    # Dynamically set summarization length
    max_len = int(input_len * 0.7)
    min_len = max(10, int(input_len * 0.5))

    # Ensure max_len is shorter than input_len
    max_len = min(max_len, input_len - 1)

    summary = summarizer(summarizer_text, max_length=max_len, min_length=min_len, num_beams=1, do_sample=False)[0]['summary_text']

The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [23]:
print(summary)

 The Pune-based insurance company is based in the city of Yerawada, Pune . The cost of the insurance is based at Bajaja Allianza General Insurance Company, Puna . The policy is based on Bajaj Allianz General Insurance Co. Ltd. The policy will be void and premium paid thereon forfeited to the Company . The bonus is an increase or addition in the Sum Insured granted by the insurer without an associated increase in premium . The company is a global health insurance provider . The insurance company has a policy based in Pune and is based there . The premium is based upon the policy amount of $46.6 million . The policies are available in India and the policy is available in the U.S. and the U-S. is available for a year . The Insurance Company has a three-month policy. The policies cover the entire policy. A policy is not available in Puna and the insurance. The insurance. is based based on the policy. It is available to cover the policy of $1,000 per year. The Policy. The Insurance. Policy.

In [24]:
prompt=get_embedding_friendly_prompt()
prompt=prompt.format(
    user_query=query,paragraph=summary
)
output=llm(prompt,max_tokens=512)
answer=output['choices'][0]["text"].strip()

print(answer)

Yes, the data in the given paragraph supports the query. The policy is based in Pune, and the cost of the insurance is based at Bajaja Allaniz General InsuraNCe Company, PuNA. The premium paid in the policy is forfeited to the Company, and the bonus is an increase or addition in the Sum Insured granted by the insurer without an associated increase in premium. The policy is valid for a year and has a three-month policy. The policies in India and the U.S. Are available for a year. The policies cover the entire policy. The insurance company has a policy based in PuNA and is available for a year in the U.S. The policy is available in PuNA, and the insurance is available to cover the policy of $1,000 per year. The policy is available on the basis of the policy, and the amount of the policy is $46.6 million. The insurance is not available in PuNA and the policy is not available in the U.S. The policy has been approved. The coverage is for the policy, and the company is a global health insura